In [1]:
import json
import requests
from urllib.parse import urlencode
import pandas as pd
import re
from datetime import datetime
from bs4 import BeautifulSoup
from random import randint,choice
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("excludeSwitches",["enable-automation"])

In [2]:
client_id = ''
client_secret = ''
redirect_uri = 'https://localhost'
code = ''
scope = 'offline_access user.read mail.read mail.readbasic mail.readwrite'
mail_folder_id = ''
refresh_token = ''

In [143]:
#getting authorization code by signing in user
url = "https://login.microsoftonline.com/consumers/oauth2/v2.0/authorize?"
headers = {
    'client_id': client_id,
    'response_type':'code',
    'scope':scope,
    'response_mode':'query',
    'redirect_uri':redirect_uri
}
s=Service(r'')
driver = webdriver.Chrome(service=s, options=chrome_options)
driver.get(url+urlencode(headers))

In [3]:
# getting access token for accessign user data by app
url = "https://login.microsoftonline.com/consumers/oauth2/v2.0/token"
data = {
    'client_id': client_id,
    'grant_type': 'authorization_code',
    'scope': scope,
    'code': code,
    'client_secret':client_secret,
    'redirect_uri':redirect_uri
}
#if you have refresh token, you dont need another authorization code to get access token, so use refresh token if access token expires
data1 = {
    'client_id': client_id,
    'grant_type': 'refresh_token',
    'scope': scope,
    'refresh_token': refresh_token,
    'client_secret':client_secret,
    'redirect_uri':redirect_uri
}
s = requests.post(url,data=data1)
access_token = s.json()['access_token']     #got the access token

In [7]:
#getting access to signed in user outlook data
def email_finder(text):
    emails = list(re.findall("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)", text))
    emails = list(set(map(lambda x:x.lower(),emails)))
    for single in list(emails):
        for sin in check:
            if sin in single:
                emails.remove(single)
                break
    excel['emails'].append(" ".join(emails))
    return emails
url = 'https://graph.microsoft.com/v1.0/me/mailFolders/'+mail_folder_id+'/messages'
headers = {
  'Authorization': access_token
}
excel = {'emails':[]}
startDate = datetime.strptime('2023-04-03','%Y-%m-%d')
lastDate = datetime.strptime('2023-04-11','%Y-%m-%d')
pagination = True
found = 0
check = ['prod.outlook.com','image001','postmaster','incedoinc','microsoft','mailer-daemon','mailchannels','bounce','google','inbound-delivery','bosmailscan','pphosted','appriver','spam','dataoncloud','pair.com']
a = []
while pagination:
    graph_result = requests.get(url=url, headers=headers).json()
    for i in graph_result['value']:
        recievedDate = datetime.strptime(i['receivedDateTime'][:10],'%Y-%m-%d')
        if recievedDate >= startDate and recievedDate <= lastDate:
            if i['from']['emailAddress']['name'] == 'Dipesh Jain':
                if i['hasAttachments']:
                    print(1)
                    #print(i['id'])
                    attach = requests.get('https://graph.microsoft.com/v1.0/me/mailFolders/'+mail_folder_id+'/messages/'+i['id']+'/attachments', headers=headers).json()
                    attach = requests.get('https://graph.microsoft.com/v1.0/me/mailFolders/'+mail_folder_id+'/messages/'+i['id']+'/attachments/'+attach['value'][0]['id']+'/$value', headers=headers)
                    attach_text = attach.text.replace("=\r\n","")
                    attach_text = attach_text.replace("\r\n"," ")
                    #print(attach_text)
                    emails = email_finder(attach_text)
                    print(emails)
                    print('*'*40)
                else:
                    print(2)
                    html = i['body']['content']
                    soup = BeautifulSoup(html,'html.parser')
                    emails = email_finder(soup.get_text(separator=" "))
                    print(emails)
                    print('*'*40)
        else:
            found = 1
            break
    if found == 1:
        break
    if '@odata.nextLink' in graph_result.keys():
        url = graph_result['@odata.nextLink']
    else:
        pagination = False
excel['emails'] = list(set(excel['emails']))
df = pd.DataFrame(excel)
with pd.ExcelWriter("MnA Details.xlsx",mode="a",engine="openpyxl",if_sheet_exists="replace") as writer:
    df = df.applymap(lambda x: ILLEGAL_CHARACTERS_RE.sub(r'', x) if isinstance(x, str) else x)
    df.to_excel(writer, sheet_name="result")

1
['neerajpaliwal@aartek.co.in']
****************************************
1
['travis.poston@synergisticit.com']
****************************************
1
['ruyi.ren@weare.am']
****************************************
1
['manik@congruentsoft.com']
****************************************
1
['behrouz.moemeni@bemopro.com']
****************************************
1
['behrouz.moemeni@bemopro.com']
****************************************
1
['rakesh.garg@allyis.com']
****************************************
1
['dave.fisher@aerieconsulting.com']
****************************************
1
['jayesh.talpade@acumenconsulting.com']
****************************************
1
['gautamsatender@acumencog.com']
****************************************
1
['jijimon.chandran@acsiatech.com']
****************************************
1
['stefanblickensdorfer@3ss.tv']
****************************************
2
['venkatachalamd@10decoders.com']
****************************************
2
['vadym@7ninjas.com']

['mike.george@aitechnologies.co']
****************************************
1
['gib@twobarrels.com', 'e1pk4y2-006idb-10@twobarrels.com']
****************************************
1
['tyreekus.peters@adeptbisoftware.com']
****************************************
1
['ferando.mendivli@amplemind.com']
****************************************
1
['haller@airdev.co']
****************************************
1
['a.modi@agilfx.com']
****************************************
2
['john.clarke@codeoneinc.com']
****************************************
2
['mwool@growthaccelerationpartners.com']
****************************************
2
['jforestall@alegohealth.com']
****************************************
2
['gottliebm@gottlieb.com']
****************************************
2
['gary@nwhcbs.com']
****************************************
2
['tony@primecarellc.com']
****************************************
2
['nimam@erad.co.in']
****************************************
1
['christopher.hotchkiss@capturerx